In [ ]:
import gradio as gr
from transformers import pipeline
import pandas as pd
import numpy as np
from pynput import keyboard
import time
import json
from pynput.keyboard import Key, Controller, KeyCode
import eng_to_ipa as ipa
import librosa

In [ ]:
import sys 
sys.path.append("./clap-ipa")
from clap.encoders import *
import torch.nn.functional as F
from transformers import DebertaV2Tokenizer, AutoProcessor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

speech_encoder = SpeechEncoder.from_pretrained('anyspeech/clap-ipa-tiny-speech')
phone_encoder = PhoneEncoder.from_pretrained('anyspeech/clap-ipa-tiny-phone')
phone_encoder.eval().to(device)
speech_encoder.eval().to(device)

tokenizer = DebertaV2Tokenizer.from_pretrained('charsiu/IPATokenizer')
processor = AutoProcessor.from_pretrained('openai/whisper-tiny')

In [ ]:
pipe = pipeline(task="zero-shot-audio-classification", model="laion/clap-htsat-fused")

In [ ]:
import time

def ipa_converted_vals(labels, audio):
    ipa_val_list = []

    for x in range(len(labels)):
        start = time.time()
        audio_input = processor(audio, return_attention_mask=True, sampling_rate=16000,max_length=32000,return_tensors='pt')
        ipa_input = tokenizer(ipa.convert(labels[x], keep_punct=True), return_token_type_ids=False, return_tensors='pt')
        with torch.no_grad():
            speech_embed = speech_encoder(**audio_input).pooler_output
            phone_embed = phone_encoder(**ipa_input).pooler_output

        similarity = F.cosine_similarity(speech_embed,phone_embed,dim=-1)
        ipa_val_list.append((similarity.cpu().detach().numpy())[0])

    return ipa_val_list

In [ ]:
# Load the macros from the JSON file
with open('macros.json', 'r') as file:
    data = json.load(file)

# Extract macros from the loaded data
macros = data["macros"]

# Creates a controller
board = Controller()

# Create a dictionary to store macros
macro_functions = {}

# Iterates over every macro
for macro in macros:
    # Macro details
    name = macro["name"]
    keycodes = macro["keycodes"]
    delay = macro["delay"]

# Defines the macro function
    macro_function = lambda keycodes=keycodes, delay=delay: [
        # Press the key according to the keycode
        (board.press(Key.enter) if keycode == Key.enter else
        board.press(Key.space) if keycode == Key.space else
        # If keycode is a single character then press that key
        board.press(keycode) if len(keycode) == 1 else
        # Else if the key is in the form <...> then press that key
        board.press(KeyCode.from_vk(int(keycode[1:-1]))),
        # Delay
        time.sleep(delay),
        # Release the key according to the key code
        board.release(Key.enter) if keycode == Key.enter else
        board.release(Key.space) if keycode == Key.space else
        # If keycode is a single character then release that key
        board.release(keycode) if len(keycode) == 1 else
        # Else if the key is in the form <...> then release that key
        board.release(KeyCode.from_vk(int(keycode[1:-1]))))
        # Iterate over every keycode in the macro
        for keycode in keycodes
    ]

    # Add the macro function to the dictionary with the macro name as the key
    macro_functions[name] = macro_function
    

In [ ]:
#Input - list of strings, list of corresponding scores
#Output - list of dictionaries where 'score' contains score corresponding to 'label' value
def joinDictionary (labels, scores):
    combinedDictionary = []
    dictionary = {'score': 0 ,'label':'test'}
    for i in range(len(labels)):
        #Create new dictionary for each label
        dictionary['score'] = scores[i]
        dictionary['label'] = labels[i]
        #add new dictionary to combined list
        combinedDictionary.append(dictionary.copy())
    return combinedDictionary

In [ ]:
#empty array 
audio_array = np.empty([])

word_detected = False

actions = ["silence"]
#scores = [0.998,0.01,0.22]
for x in macro_functions.keys():
    actions.append(x)

#actions = ["nothing","clap","snap",]


#CLAP model's prediction function
def classify(audio):
    global audio_array, word_detected, actions
    i = 0
    
    samplerate, array = audio
    
    #converts the input live audio array to numpy array and appends it to current audio array
    live_array = np.array(array)
    audio_array = np.append(audio_array, live_array)
    
    dimension = audio_array.shape
    
    
    #removes 5 second of audio from array if it's longer than 5 seconds
    if((dimension[0]) > (240000)):
        audio_array = audio_array[-240000:]
        print("Audio size: ", audio_array.shape)
    
    print(actions)

    #Pass audio to model for predictions
    result = joinDictionary(actions, ipa_converted_vals(actions, audio_array))

    print(result)
    
    #Formats the result to a dataframe and retrives the highest scoring label
    df = pd.DataFrame(result)
    max = df.idxmax(numeric_only=True)
    index = int(max[0])
    word = df.iat[index,1]
    print(df)
    print(word)
    
    #Checks if the highest scored label was an action label
    '''if word != "nothing":
        word_detected = True
        print("word was detected: ", word)
        print("pressing macros...")
        macro_functions[word]()'''

    if word != "silence":
        word_detected = True
        print("word was detected: ", word)
        print("pressing macros...")
        macro_functions[word]()
        
   
    #if word is detected or if the audio reaches 5 seconds, 
    #then reset the audio array to empty
    if (word_detected == True):
        audio_array = np.empty([1,1])
        print("All contents of audio array was removed.")
        word_detected = False
    
    return df

In [ ]:
#Settings for Gradio Interface
demo = gr.Interface(
    fn=classify,
    inputs=gr.Audio(sources=["microphone"],streaming=True),
    outputs=[gr.Dataframe()],
    live=True,
    
)

In [ ]:
#Launches Gradio Interface
demo.launch()